In [1]:
import yaml
from GeneralMethod import transforRepPattern, findElementValue, uniRep


# set variable to read the mined patterns
sam_types = [] # resource types
sam_allentries = [] # configuration entries
sam_correlationleftCorse = [] # coarse-gained dependencies: left elements
sam_correlationrightCorse = [] # coarse-gained dependencies: right elements
sam_correlationleft = [] # fine-gained dependencies: left elements
sam_correlationright = [] # fine-gained dependencies: right elements
sam_keyMapValue_key = [] # mappings about the confiration entry and values, confiration entry
sam_keyMapValue_value = [] # mappings about the confiration entry and values, confiration entry values

In [ ]:
# read the learned patterns
def readKnowledge():
    for line in open("Patterns/{}.txt".format("sam_types")):
        line = line.replace("\n","")
        sam_types.append(line)
    
    for line in open("Patterns/{}.txt".format("sam_allentries")):
        line = line.replace("\n","")
        sam_allentries.append(line)

    for line in open("Patterns/{}.txt".format("sam_correlationleftFine0.05")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationleft.append(newlist)
    for line in open("Patterns/{}.txt".format("sam_correlationrightFine0.05")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationright.append(newlist)


    for line in open("Patterns/{}.txt".format("sam_correlationleftCoarse0.05")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationleftCorse.append(newlist)
    for line in open("Patterns/{}.txt".format("sam_correlationrightCoarse0.05")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationrightCorse.append(newlist)
    
    keyMapValue = findElementValue()
    # print(sam_keyMapValue)
    for key in keyMapValue.keys():
        
        sam_keyMapValue_key.append(key)
        sam_keyMapValue_value.append(keyMapValue[key])


readKnowledge()

In [3]:
#--------------------------misconfugraiton detection----------

# 1. resoure type check
def samTypeCheck(flat_config):
    # print("resoure type check")
   
    problem_types = []
    for key_i in flat_config.keys():
        key_i_tmp = key_i.split(".")
        if key_i_tmp[-1] == "Type":
            if flat_config[key_i] not in sam_types:
                problem_types.append(flat_config[key_i])
    problem_types = list(set(problem_types))
    print("[Resource Type Errors: {}]".format(problem_types))
    print(len(problem_types))
    return "****[Resource Type Errors: {}]".format(problem_types), len(problem_types)
    

In [4]:
# 2. configuration entry check
def samEntryCheck(flat_config):
   
    problem_entries = []
    specialProcessEntry = ["Resources.PHAWS::Serverless::Function.Properties.Environment.Variables", "Resources.PHAWS::Serverless::Api.Properties.Variables", "Properties.Parameters", "Resources.PHAWS::Serverless::Api.Properties.GatewayResponses", "Resources.PHAWS::Serverless::Api.Properties.DefinitionBody", "Resources.PHAWS::Serverless::Api.Properties.Auth.Authorizers", "Resources.PHAWS::Serverless::Function.Properties.Tags", "Globals.Function.Environment.Variables"]
    specialSuffix = ["Sub", "If", "ImportValue", "GetAtt", "Join"]

    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)   

    for key_i in flat_config_key_new:
        if key_i not in sam_allentries:

            key_i_tmp = key_i.split(".")
            checkflag0 = checkListEntryInValue(key_i_tmp[-1], specialSuffix)
            
            checkflag00 = 1000 

            if len(key_i_tmp)>1:
                checkflag00 = checkListEntryInValue(key_i_tmp[-2], specialSuffix)
        
            if checkflag0 != 0:
                
                key_i_new = ".".join(key_i_tmp[0:-1])
                if checkflag00 > 0 and checkflag00 < 1000: 
                   
                    key_i_new = ".".join(key_i_tmp[0:-2])
            else:
                
                key_i_new = ".".join(key_i_tmp)
            
            checkflag1 = CustomcheckEntryInList(key_i_new, sam_allentries)
            checkflag2 = checkListEntryInValue(key_i_new, specialProcessEntry)
        
            if checkflag1 == 0 and checkflag2 == 0:
                if "Fn::Transform" in key_i_tmp:
                    if ("Fn::Transform.Name" not in ".".join(key_i_tmp)) and ("Fn::Transform.Parameters" not in ".".join(key_i_tmp)):
                        problem_entries.append(".".join(key_i_tmp)+"-----lacking Name properties")
                else:
                    problem_entries.append(".".join(key_i_tmp))

            problem_entries = list(set(problem_entries))

    problem_entries = list(set(problem_entries))
        
    print("[Configuration Entry Errors: {}]".format(problem_entries))
    print(len(problem_entries))
    return "****[Configuration Entry Errors: {}]".format(problem_entries), len(problem_entries)


def checkListEntryInValue(value, checklist):
    checkflag = 0
    for k in checklist:
        if k not in value:
            checkflag = checkflag + 0
        else:
            checkflag = checkflag + 1
    return checkflag

def CustomcheckEntryInList(value, checklist):
    # print(value)
    checkflag = 0
    value_new = value.split(".")
    for k in checklist:
        k_new = k.split(".")
        if len(k_new) == len(value_new):
            newflag = 0 
            for index in range(len(value_new)):
                if value_new[index] == k_new[index]:
                    newflag = newflag + 0
                else:
                    newflag = newflag + 1


            if newflag > 0:
                checkflag = checkflag + 0
            else:
                checkflag = checkflag + 1
    return checkflag

In [5]:
# 3. configuration entry value check
def  samKeyvalueCheck(flat_config):


    
    problem_sam_key = []
    true_sam_key = []
    problem_sam_value = []
    
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)


    for index in range(len(flat_config_key_new)):

        for key_index in range(len(sam_keyMapValue_key)):
            
            if flat_config_key_new[index] == sam_keyMapValue_key[key_index]:

                if flat_config_value_new[index] not in sam_keyMapValue_value[key_index]:
                    
                    problem_sam_key.append(flat_config_key_new[index])
                    true_sam_key.append(sam_keyMapValue_value[key_index])
                    problem_sam_value.append(flat_config_value_new[index])

    for i in range(len(problem_sam_key)):
        # print("3.3--》 Suspicious configuration entry values, specifically, please check")
        # print("3.3.1--》***Suspicious configuration entry***** {}".format(problem_sam_key[i]))
        print("--》***Suspicious values***** {}".format(problem_sam_value[i]))
    
    print(len(problem_sam_value))
    return "*****[Configuration Entry Value Errors: {}]".format(problem_sam_value), len(problem_sam_value)

In [6]:

# 4. configuration dependency check

def samRuleCheckCorse(flat_config):
    problem_sam_ruleright = []
    entryfile = ReptoEntryvalueRepSAMCorse(flat_config)
    entryfileset = set(entryfile)
   
    for index, leftvalue in enumerate(sam_correlationleftCorse):

        leftvalue_tmp = set(leftvalue)
        if leftvalue_tmp.issubset(entryfileset):

            for rightvalue in sam_correlationrightCorse[index]:
                if rightvalue not in entryfile:
                    problem_sam_ruleright.append(rightvalue)
    problem_sam_ruleright = list(set(problem_sam_ruleright))
    print("[Configuration Entry Depenency Errors: lacking {}]".format(problem_sam_ruleright))
    print(len(problem_sam_ruleright))
    return "*****[Configuration Entry Depenency Errors: lacking {}]".format(problem_sam_ruleright), len(problem_sam_ruleright)


def samRuleCheckFine(flat_config):
    problem_sam_ruleright = []
    entryfile = ReptoEntryvalueRepSAM(flat_config)
    # print(entryfile)
    entryfileset = set(entryfile)

    for index, leftvalue in enumerate(sam_correlationleft):

        leftvalue_tmp = set(leftvalue)
        if leftvalue_tmp.issubset(entryfileset):
 
            for rightvalue in sam_correlationright[index]:
                if rightvalue not in entryfile:
                    problem_sam_ruleright.append(rightvalue)
    problem_sam_ruleright = list(set(problem_sam_ruleright))
    print("[Configuration Value Depenency Errors: lacking {}]".format(problem_sam_ruleright))
    print(len(problem_sam_ruleright))
    return "****[Configuration Value Depenency Errors: lacking {}]".format(problem_sam_ruleright), len(problem_sam_ruleright)
    



def ReptoEntryvalueRepSAMCorse(flat_config):

    entryfile = []
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)
    for i in range(len(flat_config_key_new)):
        tmp = "{}".format(flat_config_key_new[i])  
        entryfile.append(tmp)
    entryfile = list(set(entryfile)) 
    return entryfile   


def ReptoEntryvalueRepSAM(flat_config):
    entryfile = []
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)
    for i in range(len(flat_config_key_new)):
        tmp = "{}={}".format(flat_config_key_new[i], flat_config_value_new[i])  
        entryfile.append(tmp)
    entryfile = list(set(entryfile)) 
    return entryfile    



In [ ]:
import pandas as pd
import os
# for the tested configuration file


# folder_path = '../TEST'
folder_path = '../TestFilesTotal'
output_file = '../Output/DD_output.csv'
header_written = False
for file_name in os.listdir(folder_path):
    if file_name.endswith('.yaml'):
        file_path = os.path.join(folder_path, file_name)
        print("------------------------------------")
        print(file_path)
        try:
            flat_config = uniRep(file_path)
            result1, num1 = samTypeCheck(flat_config)
            result2, num2 = samEntryCheck(flat_config)
            result3, num3 = samKeyvalueCheck(flat_config)
            result4, num4 = samRuleCheckCorse(flat_config)
            result5, num5 = samRuleCheckFine(flat_config)
            result = result1 + "\n" + result2 + "\n" + result3 + "\n" + result4 + "\n" + result5
            num = num1+num2+num3+num4+num5
            print("the number of final errors: {}".format(num))
            data = {
                    'Model':"DDmethod",
                    'Configuration':file_name, 
                    'Final_responses':result,
                    'TypeNum':num1,
                    'EntryNum':num2,
                    'ValueNum':num3,
                    'EntryDependencyNum':num4,
                    'ValueDependencyNum':num5,
                    'TotalNum':num
                
                }
            df = pd.DataFrame([data])
            # if not header_written:
            #     df.to_csv(output_file, mode='a', index=False, header=True)
            #     header_written = True
            # else:
            #     df.to_csv(output_file, mode='a', index=False, header=False)
            df.to_csv(output_file, mode='a', index=False, header=False)
        except Exception as e:
            print("Configuration file parse problem, please check grammer for: {}".format(e))